### InceptionV3 transfer learning

#### Import necessary libraries

In [1]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 
from keras.applications.resnet50 import ResNet50


from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob


c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Import Dataset

In [2]:


# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    oct_files = np.array(data['filenames'])
    oct_targets = np_utils.to_categorical(np.array(data['target']), 4)
    return oct_files, oct_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('OCT2017-R/train')
valid_files, valid_targets = load_dataset('OCT2017-R/valid')
test_files, test_targets = load_dataset('OCT2017-R/test')

# load list of oct names
oct_names = [item[20:-1] for item in sorted(glob("OCT2017-R/train/*/"))]

# print statistics about the dataset
print('There are %d total oct categories.' % len(oct_names))
print('There are %s total oct images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training oct images.' % len(train_files))
print('There are %d validation oct images.' % len(valid_files))
print('There are %d test oct images.'% len(test_files))

There are 4 total oct categories.
There are 7020 total oct images.

There are 5082 training oct images.
There are 938 validation oct images.
There are 1000 test oct images.


#### Pre-process the Data

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:16<00:00, 62.41it/s]


---
<a id='step5'></a>
### Utilize InceptionV3 pretrained model to predict OCT diagnosis


- [InceptionV3](https://keras.io/applications/#inceptionv3) pretrained on imagenet


In [11]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.InceptionV3.hdf5', 
                               verbose=1, save_best_only=True)

In [12]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

epochs = 2


# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers[:200]:
   layer.trainable = False
for layer in base_model.layers[200:]:
   layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:200]:
   layer.trainable = False
for layer in model.layers[200:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 5082 samples, validate on 938 samples
Epoch 1/2
 - 138s - loss: 1.2988 - val_loss: 10.5816

Epoch 00001: val_loss improved from inf to 10.58160, saving model to saved_models/weights.best.InceptionV3.hdf5
Epoch 2/2
 - 129s - loss: 0.7411 - val_loss: 2.3177

Epoch 00002: val_loss improved from 10.58160 to 2.31774, saving model to saved_models/weights.best.InceptionV3.hdf5
0 input_2 False
1 conv2d_95 False
2 batch_normalization_95 False
3 activation_95 False
4 conv2d_96 False
5 batch_normalization_96 False
6 activation_96 False
7 conv2d_97 False
8 batch_normalization_97 False
9 activation_97 False
10 max_pooling2d_5 False
11 conv2d_98 False
12 batch_normalization_98 False
13 activation_98 False
14 conv2d_99 False
15 batch_normalization_99 False
16 activation_99 False
17 max_pooling2d_6 False
18 conv2d_103 False
19 batch_normalization_103 False
20 activation_103 False
21 conv2d_101 False
22 conv2d_104 False
23 batch_normalization_101 False
24 batch_normalization_104 False
25 activ

304 activation_187 True
305 batch_normalization_188 True
306 activation_180 True
307 mixed9_1 True
308 concatenate_4 True
309 activation_188 True
310 mixed10 True
Train on 5082 samples, validate on 938 samples
Epoch 1/2
 - 129s - loss: 0.3055 - val_loss: 1.1402

Epoch 00001: val_loss improved from 2.31774 to 1.14024, saving model to saved_models/weights.best.InceptionV3.hdf5
Epoch 2/2
 - 122s - loss: 0.2434 - val_loss: 1.0737

Epoch 00002: val_loss improved from 1.14024 to 1.07375, saving model to saved_models/weights.best.InceptionV3.hdf5


#### Load Model weights

In [14]:
model.load_weights('saved_models/weights.best.InceptionV3.hdf5')

In [16]:
# get index of predicted dog breed for each image in test set
diagnosis_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(diagnosis_predictions)==np.argmax(test_targets, axis=1))/len(diagnosis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 76.6000%
